# Swarm Simulator for Outer Space

This swarm simulator will explore different swarm algorithms for an application in outer space. This approach will divide a plot into grid squares, which can then be analysed to determine and control the behaviour of each individual agent in the swarm.

In [2]:
#imports
import sympy as sym
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import deque
from scipy.spatial.distance import cdist, pdist, euclidean
from sympy import sin, cos, symbols, lambdify

from robot import Robot
from dust_devil import DustDevil

import random
import datetime
import os
import math

app = dash.Dash(__name__)

In [ ]:
#Plotly offline mode
init_notebook_mode(connected=True)

## Initialisation of Constants

In [ ]:
#initialising parameters
experiment = False
conversion_ms = 1/3.6 #km/h to m/s conversion actor
timer = 0


#Robot Parameters
robot_number = 200
robot_speed = 13*conversion_ms #m/s
robot_radius = 50 
robot_mass = 20 #kg
x_robot = []
y_robot = []
min_neighbour = []
gen_random = 0 #the random angle

#dust devil parameters
dust_time = 90 #time in seconds the dust devil will survive 
dust_speed = 150*conversion_ms #m/s
dust_radius = 100
dust_timer_count = 0
dust_devils = []

#Area Parameters
area = 1 #area in km^2 
side = 1 #each side length in m
per_square_km = 100/(60*60*24)
probability_dust = (per_square_km*area) #number of dust devils per second

#Sensor Parameters
sensor_radius = 40000

#Mean Squared Distance
MSD = []

## Physics Constants

In [ ]:
#setting physics constants
G = 1200
power = 2
R = 150
max_force = 1
multiply = 1.5

In [ ]:
#Function to initialise robots
def initialise(robot_number,length):
    '''
    Returns the list of robot objects

            Parameters:
                    robot_number (int): Number of robots
                    length (int): Length of one side of the area

            Returns:
                   robots (list): List of robot objects
    '''
    robots = []
    X,Y = random_position(robot_number,-10,10)
    for x,y in zip(X,Y):
        walk_time = int(np.random.normal(5,3,1))
        heading = random.uniform(0,2*math.pi)
        robots.append(Robot(x,y,1,robot_speed,robot_speed,robot_speed, walk_time,heading))
    return robots

In [ ]:
#Function to return randomised x,y position array
def random_position(robot_number,start,finish):
    '''
    Returns random position array of x and y

            Parameters:
                    robot_number (int): Number of robots
                    length (int): Length of one side of the area

            Returns:
                   x (integer list): List of random x positions
                   y (integer list): List of random y positions
    '''
    
    x = []
    y = []
    for i in range(int(robot_number)):
        x.append(random.randint(start,finish))
        y.append(random.randint(start,finish))
        walk_time = int(np.random.normal(20,5,1))
    return x,y

In [ ]:
#Function to return square grid of x,y positions
def grid_center(length):
    '''
    Returns square grid position array of x and y

            Parameters:
                    length (int): Length of one side of the area

            Returns:
                   x (integer list): List of x positions in the grid
                   y (integer list): List of y positions in the grid
    '''
    division = length/6
    center_x = []
    center_y = []
    for i in range(1,6):
        for j in range(1,6):
            center_x.append(division*j)
            center_y.append(division*i)
    return center_x,center_y

In [ ]:
#Function to retrieve class object positions
def positions(objects):
    '''
    Returns 2D positions of class objects

            Parameters:
                    objects (list): List of objects

            Returns:
                   x (numpy array int): List of x positions of the object
                   y (numpy array int): List of y positions of the object
    '''
    x = []
    y = []
    for i in range(len(objects)):
       x.append(objects[i].x)
       y.append(objects[i].y) 
    return np.array(x),np.array(y)

In [ ]:
#Function to check if the robot has gone over the edge and changing the value to negative if it has
def bounce(x_updated,y_updated,x_change,y_change,length):
    '''
    Returns bounced x and y position

            Parameters:
                    x_updated (float): X position after movement update
                    y_updated (float): Y position after movement update
                    x_change (float):  Change from previous X position
                    y_change (float):  Change from previous Y position
                    length (float):    Length of one side of the area

            Returns:
                   x_updated (float): Bounced X position
                   y_updated (float): Bounced Y position
    '''
    #if the x is larger than the side limit, then bounce off
    if(x_updated > length or x_updated < 0):
        x_updated = x_updated-2*x_change
    #if the y is larger than the side limit, then bounce off
    if(y_updated > length or y_updated < 0):
        y_updated = y_updated-2*y_change
    return x_updated,y_updated

In [ ]:
#Function check if there is any collisions between the robots (placeholder)
def collision_detection(swarm,dust_devils):
    positions_robot = np.array([positions(swarm)])
    position_dust = np.array([positions(dust_devils)])
    
    check = False
    return check

In [ ]:
#Function to work out the magnitude of a 2D vector
def magnitude(x,y):
    '''
    Finds the magnitude of 2 floats
    
            Parameters:
                   x (float): a float value
                   y (float): a float value
            
            Returns:
                   mag (np array): the resulting np array of magnitudes from the two given values

    '''
    mag = np.sqrt(np.square(x)+np.square(y))
    return mag

In [ ]:
#Function to work out a 2D unit vector
def unit(vector):
    '''
    Finds the unit vector of a passed vector
    
            Parameters:
                   Vector(2D numpy array): the 2D position vector
            
            Returns:
                   Unit(2D numpy array): the resulting unit vector from the given vector

    '''
    #print("Vector:", vector)
    mag = magnitude(vector[0],vector[1])
    #print("Magnitude", mag)
    
    if(vector[0] == 0):
        x = 0
    else:
        x = vector[0]/mag
    if(vector[1]==0):
        y = 0
    else:
        y = vector[1]/mag
    unit = np.array([x,y])


    
    return unit
    

In [ ]:
#Function to check if division by denominator is non zero, if it is zero, then the component is returned as zero
def division_check(numerator, denominator):
    '''
    Checks if the division is possible without zero division, if not set the component as zero
    
            Parameters:
                   Numerator (float): the numerator of the division
                 Denominator (float): the denominator of the division
            
            Returns:
                     Result (float): the result of the division

    '''

    if(denominator == 0):
        result = 0
    else:
        result = numerator/denominator
    
    return result


In [ ]:
#Function to call physics based area coverage algorithm
def physics_walk(swarm,G,power,R,max_force,multiply):
    '''
    Updates the robot position based on a physicomimetics algorithm

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None

    '''


    #print("Physics Called \n===============")
    #setting initial constants
    timestep = 1
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    minimum_distance = 0
    amount = len(swarm)
    
    #retrieving x and y positions
    x,y = positions(swarm)
    count = 0
    
    
    
    #looping through each robot in the swarm
    
    for robot in swarm:
        
        
        #initialising the forces as 0
        force = np.array([0,0])
        
        #finding the distance between the current robot and the other robots
        x_dist_unsorted = x-robot.x
        y_dist_unsorted = y-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))
        
        #calculating the minimum stance
        minimum_distance = minimum_distance + min(distance[np.nonzero(distance)])
 


        #determining the robots within 1.5R distance
        distance_local = (distance<multiply*R)
        

        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        force_change_dir = 0
        
        
        
        
        for i in range(len(x_dist)):
            
            #storing current position 
            current_position = np.array([position[0,i],position[1,i]])

            
            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])

            
            #calculating the force
            numerator = (G*(robot.mass**2))
            denominator = mag**2

            
            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)
            
            #calculating the unit vector of the position
            distance_unit = unit(current_position)
            
            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>R):
                force_change_dir = distance_unit
            
            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])
       
            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if(force_delta[0] > max_force):
                force_delta[0] = max_force
            elif(force_delta[0]<-max_force):
                force_delta[0] = -max_force
            if(force_delta[1] > max_force):
                force_delta[1] = max_force
            elif(force_delta[1]<-max_force):
                force_delta[1] = -max_force
                

            #calculating new force
            force = force+force_delta
        
        avg_min_distance = minimum_distance/len(swarm)
        
        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy
        

        
        #keeping the velocity within the maximum velocity of the robot
        if(velocity_x>robot.max_velocity):
            velocity_x = robot.max_velocity
        if(velocity_y>robot.max_velocity):
            velocity_y = robot.max_velocity
        if(velocity_x<-robot.max_velocity):
            velocity_x = -robot.max_velocity
        if(velocity_y<-robot.max_velocity):
            velocity_y = -robot.max_velocity
            
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)
        count = count+1

    return avg_min_distance
            

In [ ]:
#Function to create random walk
def random_walk(swarm):
        """
        Updates the movement of the robot by checking if the current heading needs to be reset, and then updating the position           according to the header

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None
        """
        weighting = 20
        deviation = 5
    
        for robot in swarm:
            if robot.check_walk():
                walk_time = int(np.random.normal(weighting,deviation,1))
                heading = random.uniform(0,2*math.pi)
                robot.walk_reset(walk_time,heading)
            x_change = math.cos(robot.heading)*robot.x_velocity
            y_change = math.sin(robot.heading)*robot.y_velocity
            x_updated = robot.x + x_change
            y_updated = robot.y + y_change
            #self.bounce(x_updated,y_updated,x_change,y_change,60000)
            robot.update_position(x_updated,y_updated)
            robot.update_timer()

In [ ]:
#Function to create position change for robots
def update_robots(swarm,G,power,R,max_force,multiply):
    '''
    Updates the robot position based on a chosen algorithm

            Parameters:
                   None
            
            Returns:
                   None

    '''
    #random_walk(swarm)
    min_neighbour = physics_walk(swarm,G,power,R,max_force,multiply)
    return min_neighbour

        
        


In [ ]:
G_array = [10**-10,10**-5,10**-2,10**-1,10**0,10**1,10**2,10**3,10**4,10**5,10**6]
power_array = [2,3,4,5]
R_array = [10,50,100,125,150,200,250]
multiply_array = [1,1.1,1.2,1.3,1.4,1.5,2]
robot_number_array = [10,25,50,100,250,500,1000]


#setting timestep parameter
time = 1000

current_directory = "Experiments/Random Seed/"

#for robot in robot_number_array:
robot = 1000
robot_path = (current_directory + str(robot) + " Robots/")
try:
    os.mkdir(robot_path)
except OSError:
    print ("Failure: Directory creation of %s failed" % robot_path)
else:
    print ("Success: Directory creation of %s succeeded" % robot_path)
image = ("Experiments/" + str(robot) + " Robots/Images/")
try:
    os.mkdir(image)
except OSError:
    print ("Failure: Directory creation of %s failed" % image)
else:
    print ("Success: Directory creation of %s succeeded" % image)
for i in range(len(G_array)):
    random_seed = os.urandom(8)
    random.seed(random_seed)
    swarm = initialise(robot, side)
    store_robots = np.zeros((2,robot,time+1))
    min_neighbours = []
    for j in range(time+1):
        #updating robots and storing
        store_robots[:,:,j] = positions(swarm)
        min_neighbours.append(update_robots(swarm,G_array[i],power,R,max_force,multiply))




    #Experiment number
    code = "G" + str(i)
    print(code)
    robots = (current_directory + str(robot) + " Robots/")
    path = (current_directory + str(robot) + " Robots/" + code + "/")
    try:
        os.mkdir(path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % path)
    else:
        print ("Success: Directory creation of %s succeeded" % path)



    np.save(path + "Robots.npy",store_robots)
    data = {'Values':[len(swarm),G_array[i],power,R,multiply,max_force,robot_speed,min_neighbours,random_seed]}
    constants = pd.DataFrame(data, index =['Number of Robots',
                                    'G', 
                                    'Power', 
                                    'Sensor Range', 
                                    'Multiplier',
                                    'Max Force',
                                    'Max Speed',
                                    'Minimum Distance to Neighbours',
                                    'Seed'
                                   ]) 

    constants.to_excel(path + "Constants.xlsx")

In [35]:
test = 129.96
print(round(test,1-int(math.floor(math.log10(abs(test))))))
print(round(test,1))

130.0
130.0
